In [1]:
import os
import sys
import yaml
import json
import numpy as np

with open("config.yaml", 'r') as yml_file:
    cfg = yaml.safe_load(yml_file)
    
#Get data path from config 
data_path = cfg['brian_laptop']['data_path']
mb_div = cfg['contants']['mb_div']

#Get dyype for meta file 
meta_data_file = os.path.join(data_path, cfg['data']['metadata'])
with open(meta_data_file) as f:
    meta_data = json.load(f)
   
    

In [2]:
import ray

object_store_memory = int(0.6 * ray.utils.get_system_memory() // 10 ** 9 * 10 ** 9)
object_store_memory

10000000000

In [3]:
import ray

object_store_memory = int(0.6 * ray.utils.get_system_memory() // 10 ** 9 * 10 ** 9)
ray.shutdown()
ray.init(
            include_webui=False,
            ignore_reinit_error=True,
            plasma_directory="/tmp",
            object_store_memory=object_store_memory,
        )
import modin.pandas as pd

2019-10-09 20:57:13,457	WARNING worker.py:1373 -- WARNING: Not updating worker name since `setproctitle` is not installed. Install this with `pip install setproctitle` (or ray[debug]) to enable monitoring of worker processes.
2019-10-09 20:57:13,466	INFO node.py:498 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-10-09_20-57-13_463655_54088/logs.
2019-10-09 20:57:13,579	INFO services.py:409 -- Waiting for redis server at 127.0.0.1:23282 to respond...
2019-10-09 20:57:13,700	INFO services.py:409 -- Waiting for redis server at 127.0.0.1:38089 to respond...
2019-10-09 20:57:13,706	INFO services.py:809 -- Starting Redis shard with 3.44 GB max memory.
2019-10-09 20:57:13,725	INFO node.py:512 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-10-09_20-57-13_463655_54088/logs.
2019-10-09 20:57:13,732	WARNING services.py:1363 -- WARNING: object_store_memory is not verified when plasma_directory is set.
2019-10-09 20:57:13,733	INFO services.py:14

In [4]:
import ray
import modin
print(ray.__version__)
print(modin.__version__)

ray.shutdown()
ray.init(num_cpus = 2)
import modin.pandas as pd

2019-10-07 15:44:47,160	WARNING worker.py:1373 -- WARNING: Not updating worker name since `setproctitle` is not installed. Install this with `pip install setproctitle` (or ray[debug]) to enable monitoring of worker processes.
2019-10-07 15:44:47,163	INFO node.py:498 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-10-07_15-44-47_162753_20447/logs.
2019-10-07 15:44:47,275	INFO services.py:409 -- Waiting for redis server at 127.0.0.1:50248 to respond...


0.7.3
0.6.1


2019-10-07 15:44:47,392	INFO services.py:409 -- Waiting for redis server at 127.0.0.1:59247 to respond...
2019-10-07 15:44:47,396	INFO services.py:809 -- Starting Redis shard with 3.44 GB max memory.
2019-10-07 15:44:47,412	INFO node.py:512 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-10-07_15-44-47_162753_20447/logs.


ValueError: The default object store size of 5.15 GB will use more than 90% of the available memory on this node (5.33 GB). Please reduce the object store memory size to avoid memory contention with other applications, or shut down the applications using this memory.

In [ ]:
%%time

nyc_taxi_fare_file = os.path.join(data_path, cfg['data']['kaggle_nyc'])
nyc_taxi_fare_dtype = dict(zip(meta_data['kaggle_nyc']['colnames'], meta_data['kaggle_nyc']['dtype']))

#Note it doesn't like the parse datetime here. 
nyc_taxi_fare = pd.read_table(nyc_taxi_fare_file, sep = ",",
                            dtype = nyc_taxi_fare_dtype)

In [9]:
type(nyc_taxi_fare)

modin.pandas.dataframe.DataFrame

In [10]:
%%time

nyc_taxi_fare.head()

CPU times: user 7.04 ms, sys: 3.13 ms, total: 10.2 ms
Wall time: 8.1 ms


,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,24:57.0,15.3,2011-05-28 14:24:57 UTC,-74.003957,40.707345,-73.987255,40.756599,1
1,04:00.0,10.5,2010-06-13 00:04:00 UTC,-73.985308,40.727445,-73.983152,40.756828,1
2,40:23.0,15.7,2012-06-01 21:40:23 UTC,-73.991942,40.763950,-73.994583,40.718426,1
3,56:00.0,5.0,2014-03-03 21:56:00 UTC,-73.982507,40.692082,-73.968980,40.694112,2
4,21:10.0,6.9,2009-05-07 07:21:10 UTC,-73.997176,40.749256,-73.982095,40.736297,4


In [16]:
%%time
nyc_taxi_fare.groupby(['passenger_count'])['fare_amount'].mean()

CPU times: user 44.3 ms, sys: 6.22 ms, total: 50.5 ms
Wall time: 48.1 ms


passenger_count
0     8.191667
1    11.405134
2    12.241952
3    11.260214
4    10.229519
5    11.670223
6    10.476476
Name: fare_amount, dtype: float64

In [19]:
%%time
nyc_taxi_fare.groupby(['passenger_count'])['fare_amount'].agg(count = len, 
                                                              mean = np.mean,
                                                              std = np.std,
                                                              min = np.min,
                                                              median = np.median)

CPU times: user 44.5 ms, sys: 5.08 ms, total: 49.6 ms
Wall time: 48.6 ms


,count,mean,std,min,median
passenger_count,,,,,
0,24.0,8.191667,4.095376,2.5,7.3
1,6973.0,11.405134,9.739049,2.5,8.5
2,1445.0,12.241952,13.548538,2.5,8.5
3,467.0,11.260214,10.085676,2.5,8.5
4,208.0,10.229519,8.600770,3.3,7.7
5,673.0,11.670223,9.499315,2.5,8.5
6,210.0,10.476476,7.071556,2.5,8.5


In [20]:
%%time
nyc_taxi_fare.groupby(['passenger_count'])['fare_amount'].describe()

CPU times: user 56.9 ms, sys: 6.27 ms, total: 63.2 ms
Wall time: 59.7 ms


,count,mean,std,min,25%,50%,75%,max
passenger_count,,,,,,,,
0,24.0,8.191667,4.095376,2.5,5.7,7.3,9.600,17.70
1,6973.0,11.405134,9.739049,2.5,6.0,8.5,12.900,162.75
2,1445.0,12.241952,13.548538,2.5,6.1,8.5,13.300,331.29
3,467.0,11.260214,10.085676,2.5,6.0,8.5,12.100,77.25
4,208.0,10.229519,8.600770,3.3,6.0,7.7,11.300,87.50
5,673.0,11.670223,9.499315,2.5,6.0,8.5,13.300,63.70
6,210.0,10.476476,7.071556,2.5,6.0,8.5,12.075,52.00


In [21]:
del(nyc_taxi_fare)
ray.shutdown()

## MODIN INTERESTING BITS ##

In [4]:
%%time

yellow_trip_parquet = os.path.join(data_path, cfg['data']['nyc_yellow_trip_parquet_gzip'])
yellow_trip = pd.read_parquet(yellow_trip_parquet)

CPU times: user 1.34 s, sys: 470 ms, total: 1.81 s
Wall time: 51.3 s


In [5]:
yellow_trip.head()

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,rate_code,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,surcharge,mta_tax,tip_amount,tolls_amount,total_amount
0,CMT,2010-06-18 10:43:46,2010-06-18 11:22:12,4,20.0,-73.972894,40.792810,2,0.0,-73.776308,40.645528,Cas,45.0,0.0,0.5,0.0,4.57,50.07
1,CMT,2010-06-26 15:02:57,2010-06-26 15:07:15,1,0.7,-73.987815,40.748480,1,0.0,-73.977535,40.753880,Cas,4.5,0.0,0.5,0.0,0.00,5.00
2,CMT,2010-06-24 07:36:37,2010-06-24 07:43:29,1,1.0,-74.006497,40.732921,1,0.0,-73.998122,40.725982,Cas,5.7,0.0,0.5,0.0,0.00,6.20
3,CMT,2010-06-23 18:42:21,2010-06-23 18:55:41,2,3.5,-73.987534,40.733397,1,0.0,-73.951097,40.770543,Cre,10.9,1.0,0.5,1.0,0.00,13.40
4,CMT,2010-06-23 14:10:43,2010-06-23 14:17:02,1,1.2,-73.955832,40.779476,1,0.0,-73.968117,40.765118,Cas,5.7,0.0,0.5,0.0,0.00,6.20


In [7]:
%%time
yellow_trip.groupby(['passenger_count'])['fare_amount'].describe()

To request implementation, send an email to feature_requests@modin.org.


CPU times: user 27 s, sys: 9.26 s, total: 36.2 s
Wall time: 37 s


,count,mean,std,min,25%,50%,75%,max
passenger_count,,,,,,,,
0,39.0,16.477179,14.089850,4.08,8.175,10.35,15.995,61.2
1,10207628.0,10.004302,7.960635,2.50,5.700,7.70,11.300,200.0
2,2293152.0,10.508583,8.214817,2.50,5.700,8.10,11.700,200.0
3,680684.0,10.312700,7.858859,2.50,5.700,8.10,11.700,175.3
4,323814.0,10.512960,8.147212,2.50,5.700,8.10,11.700,200.0
5,1262078.0,10.184717,7.849930,2.50,5.700,7.70,11.300,200.0
6,57696.0,9.851972,7.231171,2.50,5.700,7.70,11.300,142.1
9,1.0,11.700000,NaN,11.70,11.700,11.70,11.700,11.7
133,1.0,9.300000,NaN,9.30,9.300,9.30,9.300,9.3
